In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler

In [42]:
demanda_df = pd.read_csv(".\Datos\demanda.csv")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\juanc\AppData\Local\Temp\ipykernel_17204\1703249404.py:1: SyntaxWarning: invalid escape sequence '\D'
  demanda_df = pd.read_csv(".\Datos\demanda.csv")


In [43]:
demanda_prod_df = demanda_df[demanda_df['id_producto']==1]
demanda_prod_df['date'] = pd.to_datetime(demanda_prod_df['date'], format='%Y-%m-%d')

# Primero, asegurar que todas las fechas estan
# Los datos de demanda iniciales no tienen enero 2020
fecha_inicial = pd.to_datetime('2020-01-01')
fechas_df = pd.DataFrame({'date': pd.date_range(start=fecha_inicial, end=demanda_prod_df['date'].max())})
complete_demanda_prod_df = fechas_df.merge(demanda_prod_df
                                           ,how='left'
                                           ,left_on='date'
                                           ,right_on='date'
                                           )


complete_demanda_prod_df['anio'] = complete_demanda_prod_df['date'].dt.year
complete_demanda_prod_df['mes'] = complete_demanda_prod_df['date'].dt.month
complete_demanda_prod_df['dia'] = complete_demanda_prod_df['date'].dt.day

C:\Users\juanc\AppData\Local\Temp\ipykernel_17204\1390649101.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demanda_prod_df['date'] = pd.to_datetime(demanda_prod_df['date'], format='%Y-%m-%d')


In [44]:
# Preprocesamiento de la serie de tiempo
complete_demanda_prod_df['demanda'].ffill(inplace=True)
complete_demanda_prod_df['demanda'].bfill(inplace=True)


# Eliminacion de 29 de febrero.
complete_demanda_prod_df = complete_demanda_prod_df.sort_values(by='date', ascending=True)
complete_demanda_prod_df.reset_index(drop=True, inplace=True)

# Crear una columna indicadora para el 29 de febrero
complete_demanda_prod_df['is_feb_29'] = complete_demanda_prod_df['date'].apply(lambda x: x.month == 2 and x.day == 29)
complete_demanda_prod_df = complete_demanda_prod_df[~complete_demanda_prod_df['is_feb_29']]

C:\Users\juanc\AppData\Local\Temp\ipykernel_17204\3688260578.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  complete_demanda_prod_df['demanda'].ffill(inplace=True)
C:\Users\juanc\AppData\Local\Temp\ipykernel_17204\3688260578.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



In [45]:
# Evaluacion de la varianza para saber si necesita alguna transformacion con la prueba de Levine
antes_comp_df = complete_demanda_prod_df[complete_demanda_prod_df['date'] < '2021-07-02']
antes_comp_df.reset_index(inplace=True, drop=True)
despues_comp_df = complete_demanda_prod_df[complete_demanda_prod_df['date'] >= '2021-07-02']
despues_comp_df.reset_index(inplace=True, drop=True)

## Nota
Hacer transformaciones separando datasets antes y despues de la competencia y volver a unir.

In [46]:
# Aplicando transformacion log a datos para que la varianza no cambie
antes_comp_df['log_demanda'] = np.log(antes_comp_df['demanda'] + 1)
despues_comp_df['log_demanda'] = np.log(despues_comp_df['demanda'] + 1)


# Funcion para aplicar transformaciones (normalizacion y quizas log o raiz cuadrada)
scaler_antes = StandardScaler()
scaler_despues = StandardScaler()

antes_comp_df['log_demanda_z_scaled'] = scaler_antes.fit_transform(antes_comp_df['log_demanda'].values.reshape(-1, 1))
despues_comp_df['log_demanda_z_scaled'] = scaler_antes.fit_transform(despues_comp_df['log_demanda'].values.reshape(-1, 1))

# Funcion para crear variables exogenas
antes_comp_df['dia_de_semana'] = antes_comp_df['date'].dt.weekday.astype(int)
antes_comp_df['es_fin_de_semana'] = antes_comp_df['date'].dt.weekday.isin([5,6]).astype(int)

despues_comp_df['dia_de_semana'] = despues_comp_df['date'].dt.weekday.astype(int)
despues_comp_df['es_fin_de_semana'] = despues_comp_df['date'].dt.weekday.isin([5,6]).astype(int)

C:\Users\juanc\AppData\Local\Temp\ipykernel_17204\3342914122.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antes_comp_df['log_demanda'] = np.log(antes_comp_df['demanda'] + 1)
C:\Users\juanc\AppData\Local\Temp\ipykernel_17204\3342914122.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  despues_comp_df['log_demanda'] = np.log(despues_comp_df['demanda'] + 1)
C:\Users\juanc\AppData\Local\Temp\ipykernel_17204\3342914122.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [47]:
# Union de los datasets
preprocessed_df = pd.concat([antes_comp_df, despues_comp_df], axis=0)
preprocessed_df.drop(columns=['is_feb_29'], inplace=True)
preprocessed_df = preprocessed_df.sort_values(by='date', ascending=True)

In [48]:
# Calculo de la diferencia de variacion
preprocessed_df['demanda_diff_1_anio'] = preprocessed_df['demanda'].shift(365)
preprocessed_df.dropna(subset=['demanda_diff_1_anio'], inplace=True)
preprocessed_df.reset_index(inplace=True, drop=True)
preprocessed_df['var_rel_1_anio'] = (preprocessed_df['demanda_diff_1_anio'] - preprocessed_df['demanda']) / preprocessed_df['demanda']

preprocessed_df.drop(columns=['demanda_diff_1_anio'], inplace=True)

In [49]:
# Transformacion logaritmica a variables continuas
preprocessed_df['log_var_rel_1_anio'] = np.log(preprocessed_df['var_rel_1_anio']+1)

# Escalamiento de variables numericas
exog_var_rel_scaler = StandardScaler()
preprocessed_df['log_var_rel_1_anio_z_scaled'] = exog_var_rel_scaler.fit_transform(preprocessed_df['log_var_rel_1_anio'].values.reshape(-1, 1))

# Encoding ciclico para variables ciclicas
preprocessed_df['mes_sin'] = np.sin(2 * np.pi *preprocessed_df['mes'] / 12)
preprocessed_df['mes_cos'] = np.cos(2 * np.pi *preprocessed_df['mes'] / 12)

preprocessed_df['dia_sin'] = np.sin(2 * np.pi *preprocessed_df['dia'] / 31)
preprocessed_df['dia_cos'] = np.cos(2 * np.pi *preprocessed_df['dia'] / 31)

preprocessed_df['dia_de_semana_sin'] = np.sin(2 * np.pi *preprocessed_df['dia_de_semana'] / 7)
preprocessed_df['dia_de_semana_cos'] = np.cos(2 * np.pi *preprocessed_df['dia_de_semana'] / 7)

# Datos booleanos transformados a enteros 1 o 0
# Flag de datos despues de aparicion de tienda de la competencia
fecha_cambio = pd.Timestamp("2021-07-02")
preprocessed_df["flg_date_despues_tienda_competencia"] = (preprocessed_df["date"] >= fecha_cambio).astype(int)

preprocessed_df['es_fin_de_semana'] = preprocessed_df['es_fin_de_semana'].astype(int)


In [50]:
# Seleccion de variables finales para modelo
preprocessed_df = preprocessed_df.loc[:, ['date', 'log_demanda_z_scaled', 'es_fin_de_semana', 'log_var_rel_1_anio_z_scaled'
                                           , 'mes_sin', 'mes_cos', 'dia_sin', 'dia_cos', 'dia_de_semana_sin','dia_de_semana_cos']]

exog_cols = ['es_fin_de_semana', 'log_var_rel_1_anio_z_scaled'
             , 'mes_sin', 'mes_cos', 'dia_sin', 'dia_cos', 'dia_de_semana_sin','dia_de_semana_cos']
X_exog = preprocessed_df.loc[:, exog_cols]

## Validacion cruzada con variable endogena preprocesada y exogenas

In [ ]:
# ...